Get class-pid-index map for synapse abdominal CT dataset 

In [1]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../../dataloaders/')
import niftiio as nio

In [2]:
# lets save them in a same way as mmwhs for the ease of modifying dataloader

# normalization: cut top 2% of histogram, then doing volume-wise normalization

IMG_BNAME="/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_*.nii.gz"
SEG_BNAME="/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_*.nii.gz"






In [3]:
imgs = glob.glob(IMG_BNAME)
segs = glob.glob(SEG_BNAME)
imgs = [ fid for fid in sorted(imgs, key = lambda x: int(x.split("_")[-1].split(".nii.gz")[0])  ) ]
segs = [ fid for fid in sorted(segs, key = lambda x: int(x.split("_")[-1].split(".nii.gz")[0])  ) ]


In [4]:
imgs

['/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_0.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_1.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_2.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_3.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_4.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_5.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_6.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_7.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_8.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_9.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_10.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_11.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_12.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_13.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/image_14.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-P

In [12]:
segs

['/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_0.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_1.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_2.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_3.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_4.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_5.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_6.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_7.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_8.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_9.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_10.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_11.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_12.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_13.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/label_14.nii.gz',
 '/HDD/medical-data/ABD-CT/ABD-CT-P

In [11]:
lb = sitk.ReadImage(segs[0])
lb = sitk.GetArrayFromImage(lb)

lb.shape
# plt.imshow(lb[100, :, :], cmap='jet')
# lb[40, ...].shape
# 1 in lb[..., 40]

(148, 257, 257)

In [13]:
import json
classmap = {}
LABEL_NAME = ["BGD", "SPLEEN", "KID_R", "KID_l", "GALLBLADDER", "ESOPHAGUS", "LIVER", "STOMACH", "AORTA", "IVC", "PS_VEIN", "PANCREAS", "AG_R", "AG_L"]     

MIN_TP=1 # minimum number of true positive pixels in a slice

fid = f'/HDD/medical-data/ABD-CT/ABD-CT-Preprocessed/classmap_{MIN_TP}.json'
for _lb in LABEL_NAME:
    classmap[_lb] = {}
    for pid in range(len(segs)):
        classmap[_lb][str(pid)] = []

for pid, seg in enumerate(segs):
    lb_vol = sitk.ReadImage(seg)
    lb_vol = sitk.GetArrayFromImage(lb_vol)
    print(lb_vol.shape)
    n_slice = lb_vol.shape[0]
    for slc in range(n_slice):
        for cls in range(len(LABEL_NAME)):
            if cls in lb_vol[slc, :, :]:
                if np.sum(lb_vol[slc, :, :] == cls) >= MIN_TP:
                    classmap[LABEL_NAME[cls]][str(pid)].append(slc)
    print(f'pid {str(pid)} finished!')
    
with open(fid, 'w') as fopen:
    json.dump(classmap, fopen)
    fopen.close()  
    

(148, 257, 257)
pid 0 finished!
(140, 257, 257)
pid 1 finished!
(199, 257, 257)
pid 2 finished!
(141, 257, 257)
pid 3 finished!
(118, 257, 257)
pid 4 finished!
(132, 257, 257)
pid 5 finished!
(164, 257, 257)
pid 6 finished!
(149, 257, 257)
pid 7 finished!
(150, 257, 257)
pid 8 finished!
(149, 257, 257)
pid 9 finished!
(144, 257, 257)
pid 10 finished!
(90, 257, 257)
pid 11 finished!
(97, 257, 257)
pid 12 finished!
(125, 257, 257)
pid 13 finished!
(86, 257, 257)
pid 14 finished!
(132, 257, 257)
pid 15 finished!
(89, 257, 257)
pid 16 finished!
(90, 257, 257)
pid 17 finished!
(101, 257, 257)
pid 18 finished!
(154, 257, 257)
pid 19 finished!
(94, 257, 257)
pid 20 finished!
(145, 257, 257)
pid 21 finished!
(105, 257, 257)
pid 22 finished!
(99, 257, 257)
pid 23 finished!
(95, 257, 257)
pid 24 finished!
(185, 257, 257)
pid 25 finished!
(100, 257, 257)
pid 26 finished!
(101, 257, 257)
pid 27 finished!
(91, 257, 257)
pid 28 finished!
(196, 257, 257)
pid 29 finished!
